In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/DIALOCONAN.csv


## Install Dependencies

In [2]:
# Install dependencies
!pip install transformers datasets torch pandas numpy nltk sacrebleu rouge-score bert-score

In [3]:
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
import torch
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import bert_score
import warnings
from transformers import TrainerCallback
import logging
import os
warnings.filterwarnings("ignore")

print("Setup complete!")

2025-05-09 13:29:45.694843: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746797385.717641     138 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746797385.724722     138 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Setup complete!


## Load DialoCONAN Dataset

In [4]:
import pandas as pd
import os

# Define possible dataset paths and file names
base_paths = [
    "/kaggle/input/dialoconan",
    "/kaggle/input/dialoconan-dataset",
    "/kaggle/input/DIALOCONAN",
    "/kaggle/input"
]
file_names = [
    "DIALOCONAN.csv",
    "dialoconan.csv",
    "DIALOCONAN.CSV",
    "dialoconan.CSV"
]

# Try loading the dataset
df = None
for base_path in base_paths:
    for file_name in file_names:
        dataset_path = os.path.join(base_path, file_name)
        if os.path.exists(dataset_path):
            try:
                df = pd.read_csv(dataset_path)
                print(f"Successfully loaded dataset from: {dataset_path}")
                break
            except Exception as e:
                print(f"Error reading {dataset_path}: {e}")
    if df is not None:
        break

# If dataset not found, list available files for debugging
if df is None:
    print("Dataset not found at any specified paths. Available files in /kaggle/input:")
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))
    raise FileNotFoundError(
        "Could not find DIALOCONAN.csv. Please verify the dataset is uploaded to /kaggle/input/dialoconan/ "
        "or another directory, and check the file name for case sensitivity."
    )

# Display basic info
print("Dataset Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("Sample Data:")
print(df.head())

Successfully loaded dataset from: /kaggle/input/DIALOCONAN.csv
Dataset Shape: (16625, 6)
Columns: ['text', 'TARGET', 'dialogue_id', 'turn_id', 'type', 'source']
Sample Data:
                                                text    TARGET  dialogue_id  \
0  We’ve just imported 20k ticking time bombs fro...  MIGRANTS            0   
1  Surely people who also risked their lives to h...  MIGRANTS            0   
2  Our own must always come first. We have at lea...  MIGRANTS            0   
3  We also have at least 1 million empty homes. T...  MIGRANTS            0   
4  Our soldiers are left to rot on our streets wh...  MIGRANTS            0   

   turn_id type      source  
0        0   HS  dialo_gold  
1        1   CN  dialo_gold  
2        2   HS  dialo_gold  
3        3   CN  dialo_gold  
4        4   HS  dialo_gold  


## Preprocess the data

In [5]:
def create_dialogue_pairs(df):
    dialogues = []
    for dialogue_id in df["dialogue_id"].unique():
        dialogue = df[df["dialogue_id"] == dialogue_id].sort_values("turn_id").reset_index(drop=True)
        context = []
        for _, row in dialogue.iterrows():
            if row["type"] == "HS":
                context.append(f"[{row['TARGET']} HS]: {row['text']}")
            elif row["type"] == "CN":
                # Find previous turn using turn_id
                prev_turn = dialogue[dialogue["turn_id"] == row["turn_id"] - 1]
                hs_text = ""
                if not prev_turn.empty and prev_turn.iloc[0]["type"] == "HS":
                    hs_text = prev_turn.iloc[0]["text"]
                else:
                    logging.warning(f"Dialogue {dialogue_id}, turn_id {row['turn_id']}: No valid preceding HS turn, skipping")
                    continue
                
                # Create input-output pair
                input_text = " ".join(context[-5:]) if context else ""
                dialogues.append({
                    "input": input_text + f" [{row['TARGET']} HS]: {hs_text}",
                    "output": row["text"],
                    "target": row["TARGET"],
                    "hs_text": hs_text,
                    "cn_text": row["text"]
                })
                context.append(f"[{row['TARGET']} CN]: {row['text']}")
    return dialogues

# Create dialogue pairs
data = create_dialogue_pairs(df)
print(f"Total Dialogue Pairs: {len(data)}")

# Split data
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
test_dataset = Dataset.from_list(test_data)

print(f"Train Size: {len(train_dataset)}, Val Size: {len(val_dataset)}, Test Size: {len(test_dataset)}")

Total Dialogue Pairs: 8309
Train Size: 6647, Val Size: 831, Test Size: 831


## Tokenize Dataset

In [6]:
# Load tokenizer
model_name = "t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Tokenize function (CPU-based)
def tokenize(batch):
    with torch.no_grad():  # Disable gradient tracking
        inputs = tokenizer(
            batch["input"],
            max_length=256,
            truncation=True,
            padding="max_length",
            return_tensors="np"
        )
        outputs = tokenizer(
            batch["output"],
            max_length=64,
            truncation=True,
            padding="max_length",
            return_tensors="np"
        )
    return {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "labels": outputs.input_ids,
        "input": batch["input"],
        "output": batch["output"],
        "target": batch["target"],
        "hs_text": batch["hs_text"],
        "cn_text": batch["cn_text"]
    }

# Apply tokenization
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for training
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels", "input", "output", "target", "hs_text", "cn_text"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels", "input", "output", "target", "hs_text", "cn_text"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels", "input", "output", "target", "hs_text", "cn_text"])

# Clear any GPU memory used
import torch
torch.cuda.empty_cache()

print("Tokenization complete!")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/6647 [00:00<?, ? examples/s]

Map:   0%|          | 0/831 [00:00<?, ? examples/s]

Map:   0%|          | 0/831 [00:00<?, ? examples/s]

Tokenization complete!


## Initialize Model

In [7]:
# Clear GPU memory and set expandable segments
import torch
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

# Log GPU memory
def log_gpu_memory(step=""):
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    print(f"{step} - GPU Memory Allocated: {allocated:.2f} GiB, Reserved: {reserved:.2f} GiB")

log_gpu_memory("Before model loading")

# Load model in FP16
model_name = "t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_name).half()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

log_gpu_memory("After model loading")

print(f"Model loaded on {device}")

Before model loading - GPU Memory Allocated: 0.00 GiB, Reserved: 0.00 GiB
After model loading - GPU Memory Allocated: 1.37 GiB, Reserved: 1.39 GiB
Model loaded on cuda


## Fine-Tune T5 Model

In [ ]:
# Suppress warnings
warnings.filterwarnings("ignore", message="Some weights of")

# Set up logging
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("/kaggle/working/training_logs.txt")
    ]
)
logger = logging.getLogger(__name__)

# Clear GPU memory and set expandable segments
import torch
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

# Log initial GPU memory
def log_gpu_memory():
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    logger.info(f"GPU Memory - Allocated: {allocated:.2f} GiB, Reserved: {reserved:.2f} GiB")

log_gpu_memory()

# Custom callback for validation metrics
class CustomMetricsCallback(TrainerCallback):
    def __init__(self, tokenizer, val_dataset, max_samples=100):
        self.tokenizer = tokenizer
        self.val_dataset = val_dataset.select(range(min(max_samples, len(val_dataset))))
        self.scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

    def on_evaluate(self, args, state, control, model, **kwargs):
        logger.info(f"Computing custom metrics at step {state.global_step}")
        bleu_scores, rouge_scores, bert_scores = [], [], []
        
        model.eval()
        for item in self.val_dataset:
            try:
                input_text = item["input"]
                target = item["target"]
                
                # Generate prediction
                inputs = self.tokenizer(
                    input_text,
                    return_tensors="pt",
                    max_length=256,  # Match tokenization
                    truncation=True
                ).to(model.device)
                outputs = model.generate(
                    inputs["input_ids"],
                    max_length=64,  # Match tokenization
                    num_beams=5,
                    no_repeat_ngram_size=2,
                    early_stopping=True
                )
                pred = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                ref = item["output"]
                
                # Compute metrics
                bleu_scores.append(sentence_bleu([ref.split()], pred.split()))
                rouge_scores.append(self.scorer.score(ref, pred)["rougeL"].fmeasure)
                P, R, F1 = bert_score.score([pred], [ref], lang="en", rescale_with_baseline=True)
                bert_scores.append(F1.item())
            except Exception as e:
                logger.warning(f"Error processing item: {e}")
                continue
        
        # Log metrics
        metrics = {
            "val_bleu": np.mean(bleu_scores) if bleu_scores else 0.0,
            "val_rouge_l": np.mean(rouge_scores) if rouge_scores else 0.0,
            "val_bertscore": np.mean(bert_scores) if bert_scores else 0.0
        }
        logger.info(f"Validation Metrics: {metrics}")
        log_gpu_memory()  # Log memory after evaluation
        return None

# Training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/t5-counterspeech",
    num_train_epochs=2,  # Reduced from 3
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,  # Increased from 2
    gradient_checkpointing=True,  # Enable to save memory
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="/kaggle/working/logs",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1,
    fp16=True,
    report_to="none",
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[CustomMetricsCallback(tokenizer, val_dataset)]
)

# Train model
logger.info("Starting training...")
trainer.train()

# Save model
model.save_pretrained("/kaggle/working/t5-counterspeech-final")
tokenizer.save_pretrained("/kaggle/working/t5-counterspeech-final")

# Save logs
with open("/kaggle/working/final_training_logs.txt", "w") as f:
    f.write(str(trainer.state.log_history))

print("Training complete! Check /kaggle/working/training_logs.txt for detailed logs.")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


## Plot training/validation loss

In [ ]:
import matplotlib.pyplot as plt
logs = trainer.state.log_history
steps = [log["step"] for log in logs if "loss" in log]
losses = [log["loss"] for log in logs if "loss" in log]
plt.plot(steps, losses, label="Training Loss")
plt.xlabel("Batch Steps")
plt.ylabel("Loss")
plt.legend()
plt.show()

## Generate Counterspeech on test-split

In [ ]:
def generate_counterspeech(hs_text, target, dialogue_history=None):
    # Prepare input
    input_text = f"[{target} HS]: {hs_text}" if not dialogue_history else f"{dialogue_history} [{target} HS]: {hs_text}"
    
    # Tokenize
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=512,
        truncation=True
    ).to(device)
    
    # Generate
    outputs = model.generate(
        inputs["input_ids"],
        max_length=128,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    
    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test inference
sample_hs = "All migrants are criminals and should be deported!"
sample_target = "MIGRANTS"
sample_history = "[MIGRANTS HS]: More migrants crossing the channel today. We don’t have enough accommodation. [MIGRANTS CN]: Are you forgetting that last year every rough sleeper was offered a bed during lockdown?"
generated_cn = generate_counterspeech(sample_hs, sample_target, sample_history)
print(f"Hate Speech: {sample_hs}")
print(f"Generated Counterspeech: {generated_cn}")

## Evaluate Model

In [ ]:
def evaluate_model(dataset):
    bleu_scores = []
    rouge_scores = []
    bert_scores = []
    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    
    for item in dataset:
        # Generate prediction
        pred = generate_counterspeech(item["hs_text"], item["target"], item["input"].rsplit(f"[{item['target']} HS]:", 1)[0].strip())
        ref = item["output"]
        
        # BLEU
        bleu_scores.append(sentence_bleu([ref.split()], pred.split()))
        
        # ROUGE
        rouge_scores.append(scorer.score(ref, pred)["rougeL"].fmeasure)
        
        # BERTScore
        P, R, F1 = bert_score.score([pred], [ref], lang="en", rescale_with_baseline=True)
        bert_scores.append(F1.item())
    
    return {
        "BLEU": np.mean(bleu_scores),
        "ROUGE-L": np.mean(rouge_scores),
        "BERTScore": np.mean(bert_scores)
    }

# Run evaluation
results = evaluate_model(test_dataset)
print("Evaluation Results:")
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

## Save Results

In [ ]:
# Save evaluation results
import json
with open("/kaggle/working/evaluation_results.json", "w") as f:
    json.dump(results, f, indent=4)

# Save sample predictions
predictions = []
for item in test_dataset.select(range(5)):
    pred = generate_counterspeech(item["hs_text"], item["target"], item["input"].rsplit(f"[{item['target']} HS]:", 1)[0].strip())
    predictions.append({
        "input": item["input"],
        "hate_speech": item["hs_text"],
        "target": item["target"],
        "predicted_counterspeech": pred,
        "reference_counterspeech": item["output"]
    })

pd.DataFrame(predictions).to_csv("/kaggle/working/sample_predictions.csv", index=False)
print("Results and predictions saved!")